# Task 4.1 Demonstrate execution Modes

**Overview:** This notebook covers the available execution modes used in Qiskit. additionally it covers

*   **Execution steps** on hardware.
*   **Execution modes** like job, batch and session, and when to use each of them.
*   **Batch execution** and how to open, close and set limits on batches.


In [1]:
# Setup: Import necessary libraries
from qiskit_ibm_runtime import QiskitRuntimeService,Batch, SamplerV2 as Sampler, EstimatorV2 as Estimator
from qiskit import QuantumCircuit
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.random import random_circuit


print("Libraries imported successfully.")

Libraries imported successfully.


## Objective 1: Execute on hardware

Executing quantum circuits on real hardware involves several steps:

1. **Authentication**: Connecting to the IBM Quantum Runtime service.
2. **Backend Selection**: Choosing an available quantum processor.
3. **Circuit Preparation**: Transpiling circuits for specific hardware constraints.
4. **Job Submission**: Sending circuits for execution..
5. **Result Retrieval**: Obtain and analyze measurement outcomes.

## Objective 2: Execution modes

### Three Execution Modes

IBM Quantum provides three distinct execution modes for running quantum programs:

1. **Job Mode** (`mode=backend`)
   - Single request of Sampler or Estimator without a context manager.
   - Simplest mode for one-off executions.
   - Specify `mode=backend` when initializing the primitive. 

2. **Batch Mode** (`mode=batch`)
   - Multi-job workload made up of independently executable jobs, no condoitional relations to each other.
   - Jobs are scheduled together for efficiency.
   - Not gauranteed to run in order of submission, may run in parallel to other users jobs , QPU calibriation jobs may also run between batched jobs.
   - Queuing time doesn't decrease for the first job submitted in a job.
   - Specify `mode=batch` when initializing the primitive.

3. **Session Mode** (`mode=session`)
   - Dedicated window to run a multi-job workload.
   - execlusive access to the QPU and no other jobs can run including calibriation jobs.
   - Particularly useful for iterative tasks that require frequent communication between classical and quantum resources.
   - Queuing time doesn't decrease for the first job submitted in a job.
   - Specify `mode=session` when initializing the primitive.

## Objective 3: Choose execution mode

### When to Use Each Mode

| Mode | Best For | Key Features |
|------|----------|-------------|
| **Batch** | Large collections of independent circuits | • Entire batch scheduled together<br>• Classical computations run in parallel<br>• Minimal delay between jobs<br>• Flexible - cancel remaining jobs if needed<br>• Less expensive than sessions |
| **Session** | Variational algorithms, iterative workloads | • All batch mode functionality<br>• Dedicated access to QPU<br>• Useful when inputs arrive gradually<br>• Better for workloads requiring low latency |
| **Job** | Small experiments, quick tests | • Simplest interface<br>• Might run sooner than batch<br>• Good for learning and testing |

### Best Practices

- Use batch mode by default, especially when all inputs are ready, it is more cost-effective than sessions.

- Batch mode is ideal for submitting multiple primitive jobs at once to reduce total runtime.

- Session mode is best for iterative workflows or when dedicated QPU access is required, but it is generally more expensive.

- Job mode should be used only for a single primitive request.

- Open Plan users cannot use sessions.

### Recommendations:

- Each job has fixed overhead. If a job uses less than 1 minute of QPU time, combine multiple tasks into a single job.

- If jobs take more than 1 minute or cannot be combined, submit multiple jobs in batch or session mode to make use of parallel classical processing (up to 5 jobs), even though the QPU runs one job at a time.

- Tune workload size and parallelism specially in sessions to minimize wall-clock time and cost; splitting large jobs can be more efficient.

## Objective 4 : Run Jobs in Batch

### Batch Execution Fundamentals

Batch mode allows you to submit multiple quantum jobs that are scheduled together. This is more efficient than submitting individual jobs separately.

In [2]:
# if your account is already saved, this will load it
service = QiskitRuntimeService()
# replace <instance_name> with your instance name
# replace <token> with your IBM Quantum API token
# Uncomment and use the following line if you need to specify instance and token
# srvice = QiskitRuntimeService(channel="ibm_cloud",instance="<instance_name>",token=<token>)


### Open a batch

Can be done by initializing the `Batch` class or by using Context Manager `with Batch()`

In [3]:
# Method 1: Explicit batch creation and closure
backend = service.least_busy(operational=True, simulator=False)

#Create a batch execution context
batch = Batch(backend=backend, max_time="5m")

# Configure primitives to use batch mode
estimator = Estimator(mode=batch)
sampler = Sampler(mode=batch)

# Close the batch explicitly (no context manager used)
batch.close()

In [4]:
# Method 2: Using context manager
backend = service.least_busy(operational=True, simulator=False)
print(f"Selected backend: {backend.name}")
# Automatically handle batch lifecycle
with Batch(backend=backend, max_time="5m"):
    estimator = Estimator()  
    sampler = Sampler()

#Batch automatically closes when context exits

Selected backend: ibm_torino


Prepare circuits to run in batches

In [5]:
# Create example circuits and observables
qc1 = QuantumCircuit(2)
qc1.h(0)
qc1.cx(0, 1)
qc1.measure_all()

qc2 = QuantumCircuit(2)
qc2.h(0)
qc2.cx(0, 1)
qc2.rz(0.5, 1)
qc2.measure_all()

# Create observables for estimator
observable1 = SparsePauliOp("Z"*133)
observable2 = SparsePauliOp("X"*133)

pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit1 = pm.run(qc1)
isa_circuit2 = pm.run(qc2)
# Create primitive units (pubs) for execution
estimator_pub = (isa_circuit1, observable1)
sampler_pub = isa_circuit2

#### Batch Duration Limits

max TTL can be defined with `max_time` , it starts when the job batch starts, when max time is reached , the batch is closed and any job running will finish but all queued jobs will fail.

| Plan Type | Maximum Batch Duration |
|-----------|-----------------------|
| Open Plan | 10 minutes |
| Paid Plans | 8 hours |

Additionlly there is also interactive TTL that can not be configured (1 minute for all plans), if there is no jobs on the queue within the window, the job is temporarliy deactivated.

**Note:** These limits apply to the total execution time of all jobs within a batch.

### Close a batch

A batch automatically closes when its context manager exits. After that the batch stops accepting new jobs and finishes running all queued or active jobs until the maximum TTL is reached, then which it closes permanently.

If you are not using a context manager, you must close the batch manually. Leaving a batch open and submitting jobs later risks hitting the maximum TTL before they start, causing cancellation. Calling `batch.close()` prevents new submissions while allowing already submitted jobs to finish and their results to be retrieved.

In [6]:
with Batch(backend=backend) as batch:
    estimator = Estimator()
    sampler = Sampler()
    job1 = estimator.run([estimator_pub])
    job2 = sampler.run([sampler_pub])
 
# The batch is no longer accepting jobs but the submitted job will run to completion.
result = job1.result()
result2 = job2.result()

<span style="color:red"> **Running the cell below will use around 10-15 seconds from your IBM account credit** </span>

In [7]:
batch = Batch(backend=backend)
 
estimator = Estimator(mode=batch)
sampler = Sampler(mode=batch)
job1 = estimator.run([estimator_pub])
job2 = sampler.run([sampler_pub])
print(f"Result1: {job1.result()}")
print(f"Result2: {job2.result()}")
 
# Manually close the batch. Running and queued jobs will run to completion.
batch.close()

Result1: PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=np.ndarray(<shape=(), dtype=float64>), ensemble_standard_error=np.ndarray(<shape=(), dtype=float64>)), metadata={'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32})], metadata={'dynamical_decoupling': {'enable': False, 'sequence_type': 'XX', 'extra_slack_distribution': 'middle', 'scheduling_method': 'alap'}, 'twirling': {'enable_gates': False, 'enable_measure': True, 'num_randomizations': 'auto', 'shots_per_randomization': 'auto', 'interleave_randomizations': True, 'strategy': 'active-accum'}, 'resilience': {'measure_mitigation': True, 'zne_mitigation': False, 'pec_mitigation': False}, 'version': 2})
Result2: PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=4096, num_bits=2>)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': ExecutionSpans([DoubleSliceSpan(<start='2

In [8]:
# This will raise an error since the batch is closed
job3 = sampler.run([sampler_pub])

IBMRuntimeError: 'The session is closed.'

### Practical Example: Running Circuits in Batch

<span style="color:red"> **Running the cell below will use around 10-15 seconds from your IBM account credit** </span>

In [9]:
# Execute circuits using batch mode with context manager
with Batch(backend=backend) as batch:
    estimator = Estimator()
    sampler = Sampler()
    
    # Submit jobs to batch
    estimator_job = estimator.run([estimator_pub])
    sampler_job = sampler.run([sampler_pub])
 
# Results are available after batch context closes
# The batch is no longer accepting jobs but submitted jobs run to completion
estimator_result = estimator_job.result()
sampler_result = sampler_job.result()

print("Batch execution completed successfully!")
print(f"Estimator result: {estimator_result}")
print(f"Sampler result: {sampler_result}")

Batch execution completed successfully!
Estimator result: PrimitiveResult([PubResult(data=DataBin(evs=np.ndarray(<shape=(), dtype=float64>), stds=np.ndarray(<shape=(), dtype=float64>), ensemble_standard_error=np.ndarray(<shape=(), dtype=float64>)), metadata={'shots': 4096, 'target_precision': 0.015625, 'circuit_metadata': {}, 'resilience': {}, 'num_randomizations': 32})], metadata={'dynamical_decoupling': {'enable': False, 'sequence_type': 'XX', 'extra_slack_distribution': 'middle', 'scheduling_method': 'alap'}, 'twirling': {'enable_gates': False, 'enable_measure': True, 'num_randomizations': 'auto', 'shots_per_randomization': 'auto', 'interleave_randomizations': True, 'strategy': 'active-accum'}, 'resilience': {'measure_mitigation': True, 'zne_mitigation': False, 'pec_mitigation': False}, 'version': 2})
Sampler result: PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=4096, num_bits=2>)), metadata={'circuit_metadata': {}})], metadata={'execution': {'exe

### Batch Information and Monitoring

In [10]:
# Get batch details for monitoring
with Batch(backend=backend) as batch:
    details = batch.details()
    
print("Batch Details:")
print(f"ID: {details['id']}")
print(f"Backend: {details['backend_name']}")
print(f"State: {details['state']}")
print(f"Max Time: {details['max_time']} seconds")
print(f"Accepting Jobs: {details['accepting_jobs']}")
print(f"Mode: {details['mode']}")

Batch Details:
ID: 3f56f135-31ce-4bf0-994c-80e2558ce7d9
Backend: ibm_torino
State: open
Max Time: 600 seconds
Accepting Jobs: True
Mode: batch


### Handling Large Batches of Circuits

<span style="color:red"> **Running the cell below will use around 1 minute from your IBM account credit, you can decrease the number of circuits to decrease circuit run time.** </span>


In [ ]:
# Create a large number of circuits
max_circuits_per_batch = 10  # Limit per batch submission
total_circuits = 50
circuits = [random_circuit(3, 3) for _ in range(total_circuits)]

# Partition circuits into manageable batches
all_partitioned_circuits = []
for i in range(0, len(circuits), max_circuits_per_batch):
    all_partitioned_circuits.append(circuits[i : i + max_circuits_per_batch])

jobs = []

# Submit partitioned batches
with Batch(backend=backend):
    sampler = Sampler()
    for i, partitioned_circuits in enumerate(all_partitioned_circuits):
        pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
        isa_circuit = pm.run(partitioned_circuits)
        job = sampler.run(isa_circuit)
        jobs.append(job)
        print(f"Submitted batch {i+1} with {len(partitioned_circuits)} circuits")

print(f"\nCreated {total_circuits} circuits")
print(f"Partitioned into {len(all_partitioned_circuits)} batches of maximum {max_circuits_per_batch} circuits each")
print(f"Successfully submitted {len(jobs)} jobs to batch")

---
# Summary
---

In this notebook, we covered:

## Execution modes:

1. **Qiskit provides three execution modes** to execute quantum programs.
2. **Job mode** is the simplest interface, and it can be used for quick tests and for learning.
3. **Batch mode** can be used for independent multi job workloads, where all jobs are scheduled together for efficiency, however there is no guarantee on execution order.
4. **Session mode** can be used for iterative tasks requriing classical and quantum communication, it provides execulsive QPU access and it is more expensive that batch mode.
5. **Batch mode** can be enabled using `Batch` class or context manager `with Batch()`.


---

## Practice Questions

**1) When would you choose Session mode over Batch mode?**

A) When you have a large collection of independent circuits to run.<br/>
B) When you need the simplest interface for a one-off quantum job.<br/>
C) When running variational algorithms that require iterative communication between classical and quantum resources.<br/>
D) When you have an open plan account and need to minimize costs.<br/>

**Answer:**
<details> <br/>

C) When running variational algorithms that require iterative communication between classical and quantum resources.

Session mode is most useful when used with iterative workflows like variational quantum eigensolvers (VQE) 
as it provides dedicated QPU access. 
</details>

---

**2) What is a key limitation of Batch mode execution?**

A) Jobs in a batch cannot run in parallel with other users' jobs.<br/>
B) There's no guarantee that jobs will execute in the order they were submitted.<br/>
C) Only Open plan users can use batch mode.<br/>
D) Batch mode is more expensive than session mode.<br/>

**Answer:**
<details> <br/>

B) Jobs in batches are not gauranteed to run in order of submission, so it is important to use it only on with independently executable jobs
</details>

---

**3) What's wrong with this batch execution code?**

```
from qiskit_ibm_runtime import Batch, SamplerV2 as Sampler

# Create batch
batch = Batch(backend=backend, max_time="5m")
sampler = Sampler(mode=batch)

# Submit some jobs
job1 = sampler.run([circuit1])
job2 = sampler.run([circuit2])

# Close batch
batch.close()

# Try to submit another job
job3 = sampler.run([circuit3])

```

A) Nothing is wrong - all three jobs will execute normally.<br/>
B) You need to use a context manager (with) for batch execution.<br/>
C) The batch will automatically reopen for job3.<br/>
D) job3 will fail because the batch was already closed.<br/>

**Answer:**
<details> <br/>

D) job3 will fail because the batch was already closed. the batch will no longer accept jobs once it is closed.
</details>

---

**4) What's the most efficient way to execute this code?**

```
# Create 100 variations of the same circuit
circuits = []
for i in range(100):
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    qc.rz(i * 0.01, 1)  # Small variation
    qc.measure_all()
    circuits.append(qc)

# Need to run all circuits
```

A) Use Job mode - Submit each circuit with sampler.run([circuit]) <br/>
B) Split into groups of 10 and use Job mode for each group <br/>
C) Use Session mode - Create a session and submit circuits individually <br/>
D) Use Batch mode - Submit all circuits with sampler.run(circuits) <br/>

**Answer:**
<details> <br/>

D) Use batch mode , as the circuits are indpendent 
</details>